Incarcarea datelor de input

In [1]:
import csv
import os

crtDir =  os.getcwd()
fileName = os.path.join(crtDir, 'data', 'spam.csv')

data = []
with open(fileName) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            dataNames = row
        else:
            data.append(row)
        line_count += 1

inputs = [data[i][0] for i in range(len(data))][:100]
outputs = [data[i][1] for i in range(len(data))][:100]
labelNames = list(set(outputs))

print(inputs[:2])
print(labelNames[:2])

['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...', 'Ok lar... Joking wif u oni...']
['spam', 'ham']


Impartirea datelor in trainData and testData

In [2]:
import numpy as np

np.random.seed(5)
noSamples = len(inputs)
indexes = [i for i in range(noSamples)]
trainSample = np.random.choice(indexes, int(0.8 * noSamples), replace = False)
testSample = [i for i in indexes  if not i in trainSample]

trainInputs = [inputs[i] for i in trainSample]
trainOutputs = [outputs[i] for i in trainSample]
testInputs = [inputs[i] for i in testSample]
testOutputs = [outputs[i] for i in testSample]

print(trainInputs[:3])

['Today is \\song dedicated day..\\" Which song will u dedicate for me? Send this to all ur valuable frnds but first rply me..."', 'K tell me anything about you.', "Didn't you get hep b immunisation in nigeria."]


Extract the features

In [3]:
# # representation 1: Bag of Words
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

trainFeatures = vectorizer.fit_transform(trainInputs)
testFeatures = vectorizer.transform(testInputs)

# vocabulary size
print("vocab size: ", len(vectorizer.vocabulary_),  " words")
# no of emails (Samples)
print("traindata size: ", len(trainInputs), " emails")
# shape of feature matrix
print("trainFeatures shape: ", trainFeatures.shape)

# vocabbulary from the train data 
print('some words of the vocab: ', vectorizer.get_feature_names_out()[-20:])
# extracted features
print('some features: ', trainFeatures.toarray()[:3])

vocab size:  607  words
traindata size:  80  emails
trainFeatures shape:  (80, 607)
some words of the vocab:  ['xx' 'xxx' 'yeah' 'year' 'yes' 'yesterday' 'yo' 'you' 'your' 'yours'
 'yourself' 'yummy' 'yup' 'ě_' 'ěľ1' 'ĺł1' 'ĺł100' 'ĺł1000' 'ĺł5' 'ĺł5000']
some features:  [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


Train the model (clustering)

In [4]:
from sklearn.cluster import KMeans

unsupervisedClassifier = KMeans(n_clusters=2, random_state=0)
unsupervisedClassifier.fit(trainFeatures)

KMeans(n_clusters=2, random_state=0)

Test the model

In [5]:
computedTestIndexes = unsupervisedClassifier.predict(testFeatures)
computedTestOutputs = [labelNames[value] for value in computedTestIndexes]
for i in range(0, len(testInputs)):
    print(testInputs[i], " -> ", computedTestOutputs[i])

As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune  ->  spam
WINNER!! As a valued network customer you have been selected to receivea ĺŁ900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.  ->  spam
XXXMobileMovieClub: To use your credit, click the WAP link in the next txt message or click here>> http://wap. xxxmobilemovieclub.com?n=QJKGIGHJJGCBL  ->  spam
Oh k...i'm watching here:)  ->  spam
Did you catch the bus ? Are you frying an egg ? Did you make a tea? Are you eating your mom's left over dinner ? Do you feel my Love ?  ->  spam
Wait that's still not all that clear, were you not sure about me being sarcastic or that that's why x doesn't want to live with us  ->  spam
Great! I hope you like your man well endowed. I am  &lt;#&gt;  inches...  ->  spam
Wow. I never realized that you were so embarassed by your accomodations. I thought you liked i

Test the performance

In [6]:
from sklearn.metrics import accuracy_score

print("acc: ", accuracy_score(testOutputs, computedTestOutputs))

acc:  0.15
